### Importing Libraries

In [10]:
import numpy as np
import pandas as pd

import json

import psycopg2 as pg2
from psycopg2.extras import RealDictCursor, Json

### Collecting Fashion House Tweets

In [11]:
#Reading in application keys 

with open('../data/twitter_keys.txt') as api:
    keys = api.read().split('\n') 

#Aplication keys

import twitter, re, datetime, pandas as pd

twitter_keys = {
    'consumer_key':        keys[0],
    'consumer_secret':     keys[1],
    'access_token_key':    keys[2],
    'access_token_secret': keys[3]
}

api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret'],
    tweet_mode           =   'extended'
)

In [12]:
#Twitter API Class

class TweetMiner():
    
    result_limit = 20
    api          = None
    tweets       = []
    
    def __init__(self, keys_dict, api, result_limit = 20):
        
        self.keys_dict     = keys_dict
        self.api           = api
        self.results_limit = result_limit
        
    def mine_user_tweets(self, user='gucci', mine_retweets = False, max_pages = None, last_tweet_id = False):
        
        #time out feedback
        print("Mining tweets for:", user)
        
        tweets        = []
        last_tweet_id = False
        page           = 1
        
        while page <= max_pages:
            
            #time out feedback
            print("Mining page:", page)
            
            if last_tweet_id:
                statuses = self.api.GetUserTimeline(
                    screen_name = user,
                    count       = self.result_limit,
                    max_id      = last_tweet_id - 1
                )
            else:
                statuses = self.api.GetUserTimeline(
                    screen_name = user,
                    count       = self.result_limit
                )

            for item in statuses:
                
                last_tweet_id = item.id
                
                tweets.append({
                    'tweet_id': item.id,
                    'handle':   item.user.name,
                    'text':     item.full_text,
                    'created':  item.created_at
                })
                
            page += 1
        
        return tweets
            
        
miner = TweetMiner(twitter_keys, api, result_limit=50)

### Fashion Labels

In [13]:
wikipedia_labels = ['akris_official', 'McQueen', 'alfreddunhill', 'AlphaIndustries', 'andres_sarda', 'Anne_Fontaine',\
                   'aquascutum', 'giorgioarmani', 'emporioarmani', 'ArmaniExchange', 'AspinalofLondon', 'AspreyLondon',\
                   'AleDellAcqua', 'Dior', 'dkny', 'dolcegabbana', '_BadgleyMischka', 'BadgleyMischka', 'BALENCIAGA',\
                   'Bally', 'Balmain', 'BensonAndClegg', 'berluti', 'Beulahlondon', 'ManoloBlahnik', 'HUGOBOSS',\
                    'BottegaVeneta', 'ThomBrowneNY', 'Bulgariofficial', 'Burberry', 'NigelCabourn', 'CalvinKlein',\
                   'camillaandmarc', 'Cartier', 'Dsquared2', 'CERRUTI1881', 'CHANEL', 'Childrensalon', 'chloefashion',\
                   'matthewbridal', 'chromeheartsusa', 'ClementsRibeiro', 'Coach', 'COMMEGARCONS', 'Common Projects',\
                   'corneliajames', 'Corneliani_com', 'CuteCircuit', 'cutlerandgross', 'DegeandSkinner', 'NUMEROVENTUNO',\
                   'dkny', 'dolcegabbana', 'ermannoscervino', 'ESCADA', 'EtroOfficial', 'Faconnable', 'Fendi',\
                    'TOMFORD', 'franksorbier', 'freywilleglobal', 'STUDIO_FULTON', 'DVF', 'JPGaultier', 'GievesLondon',\
                   'givenchy', 'Goyard', 'gucci', 'HackettLondon', 'KevanHallDesign', 'halston', 'Hardy Amies ', 'HELMUTLANG',\
                   'Hermes_Paris', 'HouseofHerrera', 'TommyHilfiger', 'SherriHill', 'GeorgesHobeika', 'marcjacobs', 'jimmychoo',\
                   'johnvarvatos', 'ChristopherKane', 'Karen_Kane', 'katespadeny', 'MaryKatrantzou', ' Khaadi', 'LACOSTE',\
                   'BNYhandbags', 'KarlLagerfeld', 'DerekLamNYC', 'LANVINofficial', 'LardiniOfficial', 'guylaroche', 'Larusmiani',\
                   'JudithLeiber', '']


fashion_labels = ['gucci', 'McQueen', 'StellaMcCartney', 'Fendi', 'MaisonValentino', 'Roberto_Cavalli',\
                  'LouboutinWorld', 'EtroOfficial', 'Prada', 'dolcegabbana', 'Dior', 'LouisVuitton', 'CHANEL'\
                 'Missoni', 'OscardelaRenta', 'VeraWangGang', 'HerveLeger', 'MaisonValentino', 'Balmain',\
                 'Versace', 'giorgioarmani', 'DerekLamNYC', 'AlbertaFerretti', 'YSL', 'COMMEGARCONS', 'BALENCIAGA',\
                 'Burberry', 'TOMFORD', 'ALEXVAUTHIER', 'MIUMIUofficial', 'CalvinKlein', 'givenchy', 'TommyHilfiger',\
                 'jimmychoo', 'OffWht', 'chloefashion','dkny', 'marcjacobs', 'MichaelKors', 'Goyard', 'HUGOBOSS',\
                 'kenzo', 'HELMUTLANG', 'acnestudios', 'Bulgariofficial', 'akris_official']

prime_labels = ['gucci', 'McQueen', 'StellaMcCartney', 'Roberto_Cavalli', 'acnestudios', 'jimmychoo', 'Roberto_Cavalli',\
               'TOMFORD', 'Prada', 'akris_official']

print('wikipedia_labels', len(wikipedia_labels))
print('fashion_labels:', len(fashion_labels))
print('prime_labels:', len(prime_labels))

wikipedia_labels 96
fashion_labels: 45
prime_labels: 10


### Functions 

In [14]:
#function to connect to posgres db on ubuntu instance 

def con_cur_to_db():
    con = pg2.connect(host = '13.52.0.109', 
                     dbname = 'postgres',
                     user = 'postgres',
                     password = 'zoolander')
    cur = con.cursor(cursor_factory = RealDictCursor)
    return con, cur

In [15]:
#function to insert data into unstructured postgres db

def insert_json(data):
    con, cur = con_cur_to_db()
    for x in data:
        cur.execute('INSERT INTO all_data (data) VALUES ({});'.format(Json(x)))
    con.commit()
    con.close()

In [16]:
#grabs data from postgres db

def fetch_data(query):
    con, cur = con_cur_to_db()
    cur.execute(query)
    data = cur.fetchall()
    con.close()
    return data

In [17]:
#function to run sql queries in postgres db 

def get_handle_tweets(handle):
    query = f"""
    SELECT * 
    FROM all_data
    Where data->>'handle' = 
    '{handle}'; """
    data = fetch_data(query)
    return data

In [18]:
#transforms list of dicts of dicts to list of dicts 

def extractor(query_result):
    return [*map(lambda x: x['data'], query_result)]

### Extracting data from postgres db

In [19]:
# Extracting data from db and creating a data frame

labels = ['Acne Studios', 'Akris', 'Alexander McQueen', 'gucci', 'Jimmy Choo', 'PRADA', 'Roberto Cavalli',\
'Stella McCartney', 'TOM FORD'] 

data = []

for label in labels:
    data.extend(get_handle_tweets(label))
    
extracted_data = extractor(data)

fashion = pd.DataFrame(extracted_data)
fashion.shape

(17518, 4)

In [20]:
fashion.handle.value_counts()

gucci                3240
Stella McCartney     2000
Roberto Cavalli      2000
Acne Studios         2000
Alexander McQueen    2000
Jimmy Choo           1999
PRADA                1991
TOM FORD             1290
Akris                 998
Name: handle, dtype: int64

In [ ]:
#################################### dave function 

items = range(1, 500)

# write a function that will capture items in chunks of 10 from 5000, decending
bucket = []

def grab_10_items():
    
    # what is the lowest number in bucket so far?
    #start from there
    
    min_bucket = min(bucket)
    
    if items[min_bucket:-10] in bcuket:
        bucket.append(the stuff you find..)

grab_10_items()
